Enfoque del dataset de MAMI basado en algoritmos clásicos de aprendizaje automático

In [ ]:
### IMPORTS ###
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#from keras.utils import to_categorical

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# Cargar Dataset

In [ ]:
### GOOGLE DRIVE ###
from google.colab import drive
drive.mount('/content/drive')

PATH = "/content/drive/My Drive/TFM/Data/MAMI/"

Mounted at /content/drive


In [ ]:
### PARTICIÓN ###
df = pd.read_csv(PATH + "training.csv",  sep=",", on_bad_lines='skip', encoding='utf-8', encoding_errors='ignore', index_col=False)
df = df[['Text Transcription', 'shaming', 'stereotype', 'objectification', 'violence']]
df = df.rename(columns={"Text Transcription": "Text", "shaming": "Label1", "stereotype": "Label2", "objectification": "Label3", "violence": "Label4"})
df.fillna(" ", inplace=True)

X_train = df['Text']
y_train = pd.DataFrame({'Label1':df['Label1'], 'Label2':df['Label2'], 'Label3':df['Label3'], 'Label4':df['Label4']})

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=55, stratify=y_train)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

print('Tamaño conjunto de Entrenamiento:', len(X_train))
print('Tamaño conjunto de Evaluación:', len(X_test))

Tamaño conjunto de Entrenamiento: 6190
Tamaño conjunto de Evaluación: 1548


# Limpieza y Representación de Textos

In [ ]:
### LIMPIEZA DE TEXTOS ###
stopwords_es = stopwords.words("english")
def clean_text(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=word_tokenize(text)
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwords_es]
    # usar los stems
    tokens = [PorterStemmer().stem(word) for word in tokens]
    # eliminamos las palabras con menos de 2 caráceres
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    min_length = 2
    p = re.compile('^[a-zA-Z]+$');
    filtered_tokens=[]
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens.append(token)

    return filtered_tokens

# Label Encoding

No es necesario hacer Label Encoding

# Bolsa de Palabras

In [ ]:
### BOLSA DE PALABRAS ###
X_train = X_train.tolist()
X_test = X_test.tolist()

# entrenamos un modelo de bolsa de palabras
bow = CountVectorizer(analyzer=clean_text).fit(X_train)
# transformamos el conjunto de entrenamiento a bolsa de palabras
X_train_bow = bow.transform(X_train)
# transformamos el conjunto de evaluación a bolsa de palabras
X_test_bow = bow.transform(X_test)

print("Tamaño del vocabulario: ", len(bow.vocabulary_))

Tamaño del vocabulario:  9056


# TF-IDF

In [ ]:
### TF-IDF ###
# entrenamos un modelo tf-idf
tfidf_transformer = TfidfTransformer().fit(X_train_bow)
# transformamos el conjunto de entrenamiento
X_train_tfidf = tfidf_transformer.transform(X_train_bow)
# transformamos el conjunto de entrenamiento
X_test_tfidf = tfidf_transformer.transform(X_test_bow)

# Clasificación Clásica
Se crea un pipeline que ejecuta una secuencia de procesos:


1.   La representación de los textos en bolsa de palabras (CountVectorizer), que recibe como entrada los textos, y se les aplica dentro del CountVectorizer la función clean_text para limpiarlos y reducir el ruido.
2.   La representación en tf-idf (TfidfTransformer), recibe como entrada la salida del proceso 1, y produce los vectores tf-idf.
3. El clasificador SVC, Logistic Regression o Random Forest Clasiffier.

In [ ]:
### PIPELINE SVM ###
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('svm', MultiOutputClassifier(SVC())),
])

# Parámetros para el algoritmo SVM
grid_params_svm = [{'svm__estimator__kernel': ['linear', 'rbf'],
                    'svm__estimator__C': [0.1, 1], # [0.1, 1, 10, 100, 1000]
                    'svm__estimator__gamma':  [1, 0.1] # [1, 0.1, 0.01, 0.001, 0.0001]
                    }]
gs = GridSearchCV(pipeline, param_grid=grid_params_svm,
                  scoring='accuracy', cv=4, verbose = 1)

# entrenamos el grid
gs.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs.best_params_)
print('Mejor accuracy: %.3f' % gs.best_score_)
print(gs.best_estimator_)

best_svm = gs.best_estimator_
predictions = best_svm.predict(X_test)
print(classification_report(y_test, predictions))

Fitting 4 folds for each of 8 candidates, totalling 32 fits
Los mejores parámetros son : {'svm__estimator__C': 1, 'svm__estimator__gamma': 1, 'svm__estimator__kernel': 'linear'}
Mejor accuracy: 0.576
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7fdbd5aa69e0>)),
                ('tf', TfidfTransformer()),
                ('svm',
                 MultiOutputClassifier(estimator=SVC(C=1, gamma=1,
                                                     kernel='linear')))])
              precision    recall  f1-score   support

           0       0.70      0.15      0.24       211
           1       0.65      0.42      0.51       439
           2       0.66      0.29      0.40       362
           3       0.80      0.23      0.36       156

   micro avg       0.67      0.31      0.42      1168
   macro avg       0.70      0.27      0.38      1168
weighted avg       0.68      0.31      0.41      1168
 samples avg       0.21      0.17      0.18     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
### PIPELINE LOGISTIC REGRESSION ###
pipeline2 = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('lr', MultiOutputClassifier(LogisticRegression(random_state=0))),
])

# Parámetros para el algoritmo Logistic Regression
grid_params_lr = [{'lr__estimator__penalty': ['l1', 'l2'],
                    'lr__estimator__C': [1.0, 0.5],
                    'lr__estimator__solver':  ['liblinear']
                    }]
gs2 = GridSearchCV(pipeline2, param_grid=grid_params_lr,
                  scoring='accuracy', cv=5, verbose = 1)

# entrenamos el grid
gs2.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs2.best_params_)
print('Mejor accuracy: %.3f' % gs2.best_score_)
print(gs2.best_estimator_)

best_svm = gs2.best_estimator_
predictions = best_svm.predict(X_test)
print( classification_report(y_test, predictions))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Los mejores parámetros son : {'lr__estimator__C': 1.0, 'lr__estimator__penalty': 'l1', 'lr__estimator__solver': 'liblinear'}
Mejor accuracy: 0.569
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7fdbd5aa69e0>)),
                ('tf', TfidfTransformer()),
                ('lr',
                 MultiOutputClassifier(estimator=LogisticRegression(penalty='l1',
                                                                    random_state=0,
                                                                    solver='liblinear')))])
              precision    recall  f1-score   support

           0       0.68      0.12      0.20       211
           1       0.64      0.38      0.48       439
           2       0.70      0.25      0.36       362
           3       0.71      0.22      0.33       156

   micro avg       0.66      0.27      0.38      1168
   macro avg       0.68      0.24

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
### PIPELINE RANDOM FOREST ###
pipeline3 = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('rfc', MultiOutputClassifier(RandomForestClassifier(random_state=0))),
])

# Parámetros para el algoritmo Random Forest
grid_params_rfc = [{'rfc__estimator__criterion': ['gini', 'entropy'],
                    'rfc__estimator__max_depth': [9, 10],
                    'rfc__estimator__min_samples_split':  [10]
                    }]
gs3 = GridSearchCV(pipeline3, param_grid=grid_params_rfc,
                  scoring='accuracy', cv=5, verbose = 1)

# entrenamos el grid
gs3.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs3.best_params_)
print('Mejor accuracy: %.3f' % gs3.best_score_)
print(gs3.best_estimator_)

best_svm = gs3.best_estimator_
predictions = best_svm.predict(X_test)
print( classification_report(y_test, predictions))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Los mejores parámetros son : {'rfc__estimator__criterion': 'gini', 'rfc__estimator__max_depth': 9, 'rfc__estimator__min_samples_split': 10}
Mejor accuracy: 0.508
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7fdbd5aa69e0>)),
                ('tf', TfidfTransformer()),
                ('rfc',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=9,
                                                                        min_samples_split=10,
                                                                        random_state=0)))])
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       211
           1       0.00      0.00      0.00       439
           2       0.00      0.00      0.00       362
           3       0.00      0.00      0.00       156

   micro avg       0.00      0.00      0.00      1168
   m

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.